In [18]:
import pandas as pd
import unicodedata

# Funcție pentru eliminarea diacriticelor
def normalize_text(text):
    text = str(text).lower()
    text = unicodedata.normalize('NFKD', text)
    return ''.join(c for c in text if not unicodedata.combining(c)).strip()

# 1. Încarcă fișierele
df_xlsx = pd.read_excel("Typeform Excel Editat Model.xlsx", engine="openpyxl")
df_csv = pd.read_csv("Fillout 900 RAW Editat.csv", sep=";")

# 2. Extrage lista de simptome (coloanele 7–27 din Excel, normalizate fără diacritice)
symptom_columns = df_xlsx.columns[7:28]
symptom_map = {normalize_text(col): col for col in symptom_columns if isinstance(col, str)}
symptom_list = list(symptom_map.keys())

# 3. Normalizează textul în coloanele cu simptome din CSV
symptom_text_fields = ["Cu ce probleme de sănătate te știi?", "Prezinți unele din semnele/simptomele de mai jos"]

for col in symptom_text_fields:
    df_csv[col] = df_csv[col].fillna("").astype(str).str.lower()
    df_csv[col] = df_csv[col].str.replace(r"\s+", " ", regex=True)
    df_csv[col] = df_csv[col].str.replace("pofta de dulcefoame greu de controlat", "pofta de dulce, foame greu de controlat")
    df_csv[col] = df_csv[col].str.replace("grasime ", "grăsime ")
    df_csv[col] = df_csv[col].apply(normalize_text)

# 4. Verificăm apariția fiecărui simptom în textul normalizat
def check_symptom(row, symptom):
    for col in symptom_text_fields:
        if symptom in row[col]:
            return 1
    return 0

# 5. Creăm noile coloane pentru simptome
symptom_data = pd.DataFrame()
for symptom in symptom_list:
    symptom_data[symptom_map[symptom]] = df_csv.apply(lambda row: check_symptom(row, symptom), axis=1)

# 6. Identificăm poziția primei coloane de simptome pentru inserare
insert_at = df_csv.columns.get_loc(symptom_text_fields[0])

# 7. Eliminăm coloanele textuale originale
df_csv.drop(columns=symptom_text_fields, inplace=True)

# 8. Inserăm noile coloane de simptome în locul celor eliminate
df_result = pd.concat([df_csv.iloc[:, :insert_at], symptom_data, df_csv.iloc[:, insert_at:]], axis=1)

# 9. Exportăm fișierul final
df_result.to_csv("Fillout to Typeform Template.csv", index=False)
